In [12]:
import requests, json
from pprint import pprint
import pandas as pd
from tqdm.auto import tqdm

Functions

In [13]:

def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])
    
    return df

def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df

def get_player_fixtures(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['fixtures'])
    
    return df

In [14]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'chips': [...],
  'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_config': {...},
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 11108088}


In [15]:
pd.set_option('display.max_columns', None)
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,3,0.0,0.0,0,Fábio,0.0,1,False,Has joined Portuguese side FC Porto on loan fo...,2024-08-29T11:06:25.241953Z,54,438098.jpg,0.0,False,Ferreira Vieira,0.0,False,None,u,1,3,0,439,0,2786,2,0.0,0.0,Fábio Vieira,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,700,315,698,314,697,313,700,315,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,130,77,691,309,700,315,649,283,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-1,1,2,4,5.7,6.2,2,Gabriel,5.7,2,False,,2024-08-25T11:00:05.312580Z,69,205651.jpg,2.5,False,Fernando de Jesus,9.5,False,None,a,1,3,42,1199526,17006,428651,4205,0.8,6.1,G.Jesus,30.0,2022-07-04,594,3,2,2,5,0,0,0,4,0,0,6,151,146.8,92.4,221.0,45.7,6,3.08,0.42,3.50,5.80,248,26,215,26,77,26,185,26,NaN,,NaN,,NaN,,0.47,0.0,0.06,0.53,0.88,0.76,29,14,12,2,194,25,36,9,0.91,0.30
2,True,True,100.0,100.0,226597,0,0,4,-4,2,2,3.8,4.3,2,Gabriel,3.8,3,True,,2024-11-27T10:30:06.325759Z,64,226597.jpg,4.6,False,dos Santos Magalhães,29.1,False,None,a,1,3,83,3829507,7332,2216313,6015,0.6,13.0,Gabriel,30.0,2020-09-01,1532,3,2,6,16,0,0,0,2,0,0,7,299,402.4,130.0,251.0,78.4,18,2.42,0.92,3.34,16.13,40,13,168,37,62,3,75,11,NaN,,NaN,,NaN,,0.14,0.0,0.05,0.19,0.95,0.94,50,3,53,15,28,3,9,2,1.06,0.35
3,True,True,75.0,75.0,219847,0,0,-1,1,1,4,2.5,2.8,0,Kai,3.3,4,False,Illness - 75% chance of playing,2025-01-02T08:00:08.350130Z,79,219847.jpg,4.4,False,Havertz,9.1,False,None,d,1,3,74,2417542,446,2991742,11751,0.4,9.4,Havertz,80.0,2023-06-28,1480,7,2,6,16,0,0,0,3,0,0,10,283,372.2,220.2,559.0,115.2,17,7.77,1.34,9.11,15.63,53,7,98,7,12,5,22,6,NaN,,NaN,,3.0,,0.47,0.0,0.08,0.55,0.95,0.97,13,5,72,11,39,11,40,10,1.03,0.36
4,True,False,0.0,0.0,463748,0,0,0,0,0,1,0.0,0.0,0,Karl,0.0,5,False,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,40,463748.jpg,0.0,False,Hein,0.0,False,None,u,1,3,0,0,0,4296,7,0.0,0.0,Hein,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,526,61,506,42,470,29,526,61,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,636,59,463,56,534,61,567,68,0.00,0.00


In [16]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

In [17]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],73
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],239
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],319
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],78


In [18]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name', 'total_points','bps', 'ict_index']].head()

,first_name,second_name,name,total_points,bps,ict_index
0,Fábio,Ferreira Vieira,Arsenal,0,0,0.0
1,Gabriel,Fernando de Jesus,Arsenal,42,151,45.7
2,Gabriel,dos Santos Magalhães,Arsenal,83,299,78.4
3,Kai,Havertz,Arsenal,74,283,115.2
4,Karl,Hein,Arsenal,0,0,0.0


In [19]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
df[
   ['first_name', 'second_name', 'team_name', 'total_points','bps', 'ict_index']
].head()

,first_name,second_name,team_name,total_points,bps,ict_index
0,Fábio,Ferreira Vieira,Arsenal,0,0,0.0
1,Gabriel,Fernando de Jesus,Arsenal,42,151,45.7
2,Gabriel,dos Santos Magalhães,Arsenal,83,299,78.4
3,Kai,Havertz,Arsenal,74,283,115.2
4,Karl,Hein,Arsenal,0,0,0.0


Player Gameweek History

In [20]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


Points table of gameweek points for all weeks

In [21]:
tqdm.pandas()

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()


,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,ep_next,ep_this,event_points,first_name,form,id_player,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,name,singular_name_short
0,True,False,0.0,0.0,438098,0,0,-1,1,0,0.0,0.0,0,Fábio,0.0,1,False,Has joined Portuguese side FC Porto on loan fo...,2024-08-29T11:06:25.241953Z,54,438098.jpg,0.0,False,Ferreira Vieira,0.0,False,None,u,3,0,439,0,2786,2,0.0,0.0,Fábio Vieira,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,700,315,698,314,697,313,700,315,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,130,77,691,309,700,315,649,283,0.00,0.00,Arsenal,MID
1,True,True,100.0,100.0,205651,0,0,-1,1,2,5.7,6.2,2,Gabriel,5.7,2,False,,2024-08-25T11:00:05.312580Z,69,205651.jpg,2.5,False,Fernando de Jesus,9.5,False,None,a,3,42,1199526,17006,428651,4205,0.8,6.1,G.Jesus,30.0,2022-07-04,594,3,2,2,5,0,0,0,4,0,0,6,151,146.8,92.4,221.0,45.7,6,3.08,0.42,3.50,5.80,248,26,215,26,77,26,185,26,NaN,,NaN,,NaN,,0.47,0.0,0.06,0.53,0.88,0.76,29,14,12,2,194,25,36,9,0.91,0.30,Arsenal,FWD
2,True,True,100.0,100.0,226597,0,0,4,-4,2,3.8,4.3,2,Gabriel,3.8,3,True,,2024-11-27T10:30:06.325759Z,64,226597.jpg,4.6,False,dos Santos Magalhães,29.1,False,None,a,3,83,3829507,7332,2216313,6015,0.6,13.0,Gabriel,30.0,2020-09-01,1532,3,2,6,16,0,0,0,2,0,0,7,299,402.4,130.0,251.0,78.4,18,2.42,0.92,3.34,16.13,40,13,168,37,62,3,75,11,NaN,,NaN,,NaN,,0.14,0.0,0.05,0.19,0.95,0.94,50,3,53,15,28,3,9,2,1.06,0.35,Arsenal,DEF
3,True,True,75.0,75.0,219847,0,0,-1,1,1,2.5,2.8,0,Kai,3.3,4,False,Illness - 75% chance of playing,2025-01-02T08:00:08.350130Z,79,219847.jpg,4.4,False,Havertz,9.1,False,None,d,3,74,2417542,446,2991742,11751,0.4,9.4,Havertz,80.0,2023-06-28,1480,7,2,6,16,0,0,0,3,0,0,10,283,372.2,220.2,559.0,115.2,17,7.77,1.34,9.11,15.63,53,7,98,7,12,5,22,6,NaN,,NaN,,3.0,,0.47,0.0,0.08,0.55,0.95,0.97,13,5,72,11,39,11,40,10,1.03,0.36,Arsenal,FWD
4,True,False,0.0,0.0,463748,0,0,0,0,0,0.0,0.0,0,Karl,0.0,5,False,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,40,463748.jpg,0.0,False,Hein,0.0,False,None,u,3,0,0,0,4296,7,0.0,0.0,Hein,NaN,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,526,61,506,42,470,29,526,61,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,636,59,463,56,534,61,567,68,0.00,0.00,Arsenal,GKP


In [22]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)

100%|██████████| 709/709 [00:44<00:00, 15.88it/s]


In [23]:
# get top scoring players
points.groupby(
    ['element', 'web_name']
).agg(
    {'total_points':'sum', 'goals_scored':'sum', 'assists':'sum'}
).reset_index(
).sort_values(
    'total_points', ascending=False
).head()

,element,web_name,total_points,goals_scored,assists
327,328,M.Salah,202,17,13
181,182,Palmer,150,13,7
98,99,Mbeumo,133,13,3
350,351,Haaland,120,16,1
400,401,Isak,117,13,4


In [24]:
future_fixtures = pd.DataFrame(columns=['id_player','team_h', 'team_a', 'kickoff_time', 'is_home','difficulty', 'event'])

for i in range(1,points['id_player'].max()):
    try:
        df  = get_player_fixtures(i)
        df['id_player'] = i
        df = df[['id_player','team_h', 'team_a', 'kickoff_time', 'is_home','difficulty','event']]
        future_fixtures = pd.concat([future_fixtures, df])
    except KeyError:
        print("couldn't find key for player_id")

In [25]:
future_fixtures['opponent_team'] = future_fixtures.apply(lambda row: row['team_a'] if row['is_home'] else row['team_h'], axis=1)

In [26]:
# join player positions
future_fixtures = future_fixtures.merge(
    players,
    left_on='id_player',
    right_on='id_player'
)

In [27]:
#get cumalatice sum of points for each player ordered by rounds

# Sort by 'round'
points = points.sort_values(by='round')

# Group by 'a' and calculate cumulative sum of 'points'
points['cumulative_points'] = points.groupby('id_player')['total_points'].cumsum()

print(points.sort_values(by = 'id_player'))

      id_player      web_name  element  fixture  opponent_team  total_points  \
11            1  Fábio Vieira        1      111             16             0   
12            1  Fábio Vieira        1      129             19             0   
3             1  Fábio Vieira        1       39             18             0   
18            1  Fábio Vieira        1      182              4             0   
4             1  Fábio Vieira        1       47             13             0   
...         ...           ...      ...      ...            ...           ...   
1995        707   Rees-Dottin      707      185              9             0   
7486        708  Wilson-Brown      708      187             13             0   
7487        708  Wilson-Brown      708      191              2             0   
8076        709         Danns      709      190             19             0   
8077        709         Danns      709      196             14             0   

      was_home          kickoff_time  t

In [28]:
#get max cumulative points per player and append to fixtures

max_cumulative_points = points.groupby('id_player')['cumulative_points'].max().reset_index()

#join to future fixtures
future_fixtures = future_fixtures.merge(
    max_cumulative_points,
    left_on='id_player',
    right_on='id_player'
)

In [29]:
past_data = points[['id_player', 'opponent_team','was_home', 'cumulative_points','bps', 'total_points']].rename(columns={'was_home': 'is_home'})

future_data = future_fixtures[['id_player', 'opponent_team', 'is_home', 'cumulative_points','bps', 'total_points','event']]